In [72]:
#导入预训练模型
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
torch.save(model.state_dict(), 'resnet50_cifar10.pth')

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /home/featurize/.cache/torch/hub/v0.10.0.zip


In [196]:
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50
from torchvision import transforms
from PIL import Image
import numpy as np

# 加载预训练的 ResNet-50 模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        resnet_model = resnet50(pretrained=True)
        self.features = nn.Sequential(*list(resnet_model.children())[:-1])
        self.classifier = nn.Linear(2048, 10) 

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# PGD 攻击
def pgd_attack(model, images, labels, epsilon, alpha, num_iter):
    images.requires_grad = True
    for _ in range(num_iter):
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        grad = torch.autograd.grad(loss, images)[0]
        images = images + alpha * grad.sign()
        images = torch.clamp(images, 0, 1)
        images = torch.clamp(images, images - epsilon, images + epsilon)
    return images.detach()

# 载入模型
model = Net()
model.load_state_dict(torch.load('resnet50_cifar10.pth'), strict=False)
model.eval()

#载入图像
image_path = 'cat.png'
input_image = Image.open(image_path).convert("RGB")
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])
input_tensor = transform(input_image).unsqueeze(0)  # 添加 batch 维度

#初始化模型和优化器
model.cuda()
input_tensor = input_tensor.cuda()

#对抗样本参数
epsilon = 0.03
alpha = 0.01
num_iter = 10

#生成对抗样本
adversarial_image = pgd_attack(model, input_tensor, label, epsilon, alpha, num_iter)

#在原始图像和对抗样本上进行预测
output_original = model(input_tensor)
output_adversarial = model(adversarial_image)

_, predicted_original = torch.max(output_original.data, 1)
_, predicted_adversarial = torch.max(output_adversarial.data, 1)

class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
print(f"Original Prediction: {class_labels[predicted_original.item()]}")
print(f"Adversarial Prediction: {class_labels[predicted_adversarial.item()]}")

#保存生成的对抗样本
array = adversarial_image.squeeze().cpu().numpy()
array = (array * 255).astype(np.uint8)
array = np.transpose(array, (1, 2, 0))
image = Image.fromarray(array, mode='RGB')
image.save('adv_cat.png')


Original Prediction: cat
Adversarial Prediction: truck
